In [1]:
from dexcom_analysis_lib import *

display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_rows=1000

In [2]:
de = TidepoolExplorer()

In [14]:
# print(len(list(de.cbg_data())))

conversion = 18.018018

for d in de.cbg_data()[:999]:
    v = d['value']
    rtv = d['payload']['realTimeValue']
    sv = d['payload'].get('smoothedValue')
    calc = round(v * conversion)
    test = calc - sv if sv else None
    
    print("{} {} {} {}\n".format(v, sv, calc, test))

6.60539 None 119 None

6.77191 122 122 0

6.93843 125 125 0

7.16046 129 129 0

7.27148 131 131 0

7.27148 131 131 0

7.16046 129 129 0

6.93843 125 125 0

6.71641 121 121 0

6.43887 116 116 0

6.27235 113 113 0

5.99481 108 108 0

5.55075 100 100 0

4.82915 87 87 0

4.21857 76 76 0

3.77451 None 68 None

3.55248 64 64 0

3.33045 60 60 0

3.10842 56 56 0

3.10842 56 56 0

3.33045 60 60 0

3.83002 69 69 0

4.32958 78 78 0

4.82915 87 87 0

5.2177 None 94 None

5.27321 95 95 0

5.32872 96 96 0

5.43973 98 98 0

5.66176 102 102 0

5.99481 108 108 0

6.54988 118 118 0

7.21597 130 130 0

7.82655 141 141 0

8.32612 150 150 0

8.71467 157 157 0

9.15873 165 165 0

9.60279 173 173 0

9.99135 None 180 None

9.93584 179 179 0

9.82482 177 177 0

9.60279 173 173 0

9.32526 168 168 0

9.04772 163 163 0

8.77018 158 158 0

8.65917 156 156 0

8.38163 151 151 0

7.99308 144 144 0

7.49351 135 135 0

6.88293 124 124 0

6.10582 110 110 0

5.32872 96 96 0

4.66263 84 84 0

4.32958 78 78 0

4.27408 77 7

In [ ]:
in_range = (90, 150)
hypo_range = 54
roc_pct = 95

window_days = 5
backward_window_days = int(window_days / 2)
forward_window_days = window_days - backward_window_days

window = pd.Timedelta('{} days'.format(window_days))
forward_window = pd.Timedelta('{} days'.format(forward_window_days))
backward_window = pd.Timedelta('{} days'.format(backward_window_days))

one_day = pd.Timedelta('1 days')

plot_start_date = min(de.bg_data['Timestamp']) #'2019-08-12'
plot_start_date = pd.Timestamp(pd.to_datetime(plot_start_date).date())

plot_end_date = max(de.bg_data['Timestamp']) #'today'
plot_end_date = pd.Timestamp(pd.to_datetime(plot_end_date).date() + one_day)

plot_window_mask = (de.bg_data['Timestamp'] > plot_start_date) & (de.bg_data['Timestamp'] <= plot_end_date)
plot_window_data = de.bg_data.loc[plot_window_mask]

offset = (forward_window - backward_window) / 2

end_date = plot_end_date

all_stats = []

while True:
    start_date = end_date - one_day
    if start_date < plot_start_date: break
        
    window_start_date = start_date - backward_window
    window_end_date = start_date + forward_window
        
    date_mask = (plot_window_data['Timestamp'] > window_start_date) & (plot_window_data['Timestamp'] <= window_end_date)
    day_data = plot_window_data.loc[date_mask]
    
    bg_values = day_data["Glucose Value (mg/dL)"]
    
    if len(bg_values) > 0:
        hypo_points = [x for x in bg_values if x < hypo_range]
        low_points = [x for x in bg_values if x < in_range[0]]
        high_points = [x for x in bg_values if x > in_range[1]]
        
        n_points = len(bg_values)
        n_hypo = len(hypo_points)
        n_low = len(low_points)
        n_high = len(high_points)
        n_in_range = n_points - n_low - n_high
        
        pct_hypo = 100 * n_hypo / n_points
        pct_low = 100 * n_low / n_points
        pct_high = 100 * n_high / n_points
        pct_in_range = 100 * n_in_range / n_points
        
        abs_rate_of_change = day_data["derivative"].dropna().abs()
        percentile_rate_of_change = np.percentile(abs_rate_of_change, roc_pct)

        row = {
            'date': start_date + offset,
            '10%': np.percentile(bg_values, 10),
            '90%': np.percentile(bg_values, 90),
            'mean_rate_of_change': abs_rate_of_change.mean(),
            'pct_in_range': pct_in_range,
            'pct_hypo': pct_hypo,
            '{}%_rate_of_change'.format(roc_pct): np.percentile(abs_rate_of_change, roc_pct)
        }

        for k, v in bg_values.describe().items():
            row[k] = v

        all_stats.append(row)
    
    end_date = start_date
    
all_stats = pd.DataFrame(data=all_stats)

In [ ]:
de.plot_summary(plot_window_data, all_stats)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=plot_window_data['Timestamp'],
                y=plot_window_data["derivative"],
                name='points',
                line_color='black',
                opacity=0.5))

fig.update_layout(title_text="Blood Glucose Rate of Change",
                  xaxis_rangeslider_visible=True,
                  template="plotly_white",
                  yaxis_range=[-10,10],
                  yaxis_dtick=2,
                  xaxis_title="Date",
                  yaxis_title="BG Rate of Change (mg/dL/min)"
                 )
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=all_stats.date,
                y=all_stats['std'],
                name='std',
                line_color='black',
                opacity=0.5))

fig.update_layout(title_text="Blood Glucose Standard Deviation",
                  xaxis_rangeslider_visible=True,
                  template="plotly_white",
                  yaxis_range=[30,70],
                  yaxis_dtick=5,
                  xaxis_title="Date",
                  yaxis_title="BG Std. Dev. (mg/dL)"
                 )
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=all_stats.date,
                y=all_stats['{}%_rate_of_change'.format(roc_pct)],
                name='{}%_rate_of_change'.format(roc_pct),
                line_color='black',
                opacity=0.5))

fig.update_layout(title_text="Blood Glucose {}th Percentile Rate of Change".format(roc_pct),
                  xaxis_rangeslider_visible=True,
                  template="plotly_white",
                  yaxis_range=[1.5,3],
                  yaxis_dtick=0.25,
                  xaxis_title="Date",
                  yaxis_title="BG {}th Pctle. Rate of Change (mg/dL/min)".format(roc_pct)
                 )
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=all_stats.date,
                y=all_stats['pct_in_range'],
                name='pct_in_range',
                line_color='black',
                opacity=0.5))

fig.update_layout(title_text="Blood Glucose Percent in Range ({}–{} mg/dL)".format(in_range[0], in_range[1]),
                  xaxis_rangeslider_visible=True,
                  template="plotly_white",
                  yaxis_range=[25,75],
                  yaxis_dtick=5,
                  xaxis_title="Date",
                  yaxis_title="BG Percent in Range"
                 )
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=all_stats.date,
                y=all_stats['pct_hypo'],
                name='pct_hypo',
                line_color='black',
                opacity=0.5))

fig.update_layout(title_text="Blood Glucose Percent Hypoglycemic (< {} mg/dL)".format(hypo_range),
                  xaxis_rangeslider_visible=True,
                  template="plotly_white",
                  yaxis_range=[-0.1,0.5],
                  yaxis_dtick=0.1,
                  xaxis_title="Date",
                  yaxis_title="BG Percent Hypoglycemic".format(hypo_range)
                 )
fig.show()